In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import KNNImputer

from geopy.distance import geodesic

from datetime import datetime

In [ ]:
# Carga el dataset en un dataframe
df = pd.read_csv('uber_fares.csv')

# Nos quedamos con las filas cuya variable objetivo es un valor posible
df = df[df['fare_amount'] > 0] # Elimina el 0.01% de los datos

# Separa el 80% para train y 20% para test
train, test= train_test_split(df, test_size=0.2, random_state=1)

train.info()

In [ ]:
# Asigna el tipo de datos correcto a las variables que representan fechas
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['date'] = pd.to_datetime(train['date'])

## Limpieza

### Análisis de valores nulos
Encontramos un valor nulo de *dropoff_longitude* y otro de *dropoff_latitude*, veamos las filas donde ocurren:

In [ ]:
train[train.isnull().any(axis=1)]

Ambos valores nulos se dan en la misma fila por lo que no podemos imputar la ubicación de destino ni estimar ninguna distancia recorrida, por lo tanto siendo que es una unica fila que representa menos del 0.0006% de los datos, procedemos a eliminarla.

In [ ]:
train = train.dropna()

### Análisis de valores absurdos

#### Variables de coordenadas

* pickup_latitude
* pickup_longitude
* dropoff_latitude
* dropoff_longitude


In [ ]:
def coordenadas_absurdas(viaje):
    yield not -90 <= viaje['pickup_latitude'] <= 90, 'pickup_latitude'
    yield not -180 <= viaje['pickup_longitude'] <= 180, 'pickup_longitude'
    yield not -90 <= viaje['dropoff_latitude'] <= 90, 'dropoff_latitude'
    yield not -180 <= viaje['dropoff_longitude'] <= 180 , 'dropoff_longitude'

def procesar_coordenadas_absurdas(df):
    '''
    Procesa las coordenadas absurdas en el dataframe.
    Si una fila tiene más de una coordenada absurda, se elimina.
    Si tiene una sola coordenada absurda, se imputa mediante KNN.
    Si imputar == False, se eliminan las filas con al menos una coordenada absurda.
    '''
    def procesar_fila(viaje):
        coordenada = None
        for absurda, coord in coordenadas_absurdas(viaje):
            if absurda:
                if coordenada:  # Más de una coordenada absurda
                    return None # Convierte la fila en NaNs para posterior eliminación
                coordenada = coord

        if coordenada:
            viaje[coordenada] = np.nan # Coordenada absurda -> NaN para posterior imputación
        return viaje

    df = df.apply(procesar_fila, axis=1)
    df = df.dropna(how='all') # Elimina filas de NaNs

    # Imputar coordenada absurda mediante KNN
    imputer = KNNImputer(n_neighbors=5)
    df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']] = imputer.fit_transform(df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']])
    
    return df

train = procesar_coordenadas_absurdas(train)
test = procesar_coordenadas_absurdas(test)

#### Null Island
Isla ficticia, ubicada en 0°N 0°E, que los GPS suelen utilizar como ubicación por defecto cuando no pueden determinar la ubicación real, es decir, una forma de representar una ubicación nula.

In [ ]:
train = train[
    ~(
        (train['pickup_latitude'] == 0) & (train['pickup_longitude'] == 0) |
        (train['dropoff_latitude'] == 0) & (train['dropoff_longitude'] == 0)
    )
]

test = test[
    ~(
        (test['pickup_latitude'] == 0) & (test['pickup_longitude'] == 0) |
        (test['dropoff_latitude'] == 0) & (test['dropoff_longitude'] == 0)
    )
]

In [ ]:
def imputar_distancia(viaje):
    '''
    Calcula la distancía  en kilometros del viaje
    mediante una combinación de distancia Manhattan con
    distancia geodésica (teniendo en cuenta la curvatura
    de la Tierra.)
    '''
    lat1 = viaje['pickup_latitude']
    lon1 = viaje['pickup_longitude']
    lat2 = viaje['dropoff_latitude']
    lon2 = viaje['dropoff_longitude']

    distancia_lat = np.float32(geodesic((lat1, lon1), (lat2, lon1)).kilometers)
    distancia_lon = np.float32(geodesic((lat2, lon1), (lat2, lon2)).kilometers)

    return distancia_lat + distancia_lon

train['distance'] = train.apply(imputar_distancia, axis=1)
test['distance'] = test.apply(imputar_distancia, axis=1)

In [ ]:
train[[train['distance'] >= 200],[train['is_water'] == True]]

In [ ]:
train.info()

#### Hasta acá removimos el 2.025% de los datos

In [ ]:
train[train['distance'] > 50]

In [ ]:
sns.scatterplot(data=train, x='distance', y='fare_amount')

In [ ]:
train['fare_amount'].plot.kde()

In [ ]:
from global_land_mask import globe

train['is_water'] = train.apply(
    lambda viaje:
    globe.is_ocean(viaje['pickup_latitude'],viaje['pickup_longitude'])
    | globe.is_ocean(viaje['dropoff_latitude'],viaje['dropoff_longitude']),
    axis=1)

In [ ]:
train[(train['is_water'] == True)].count()